# How to use `locals`
`locals` is a pure Python package that ingests JWST Wide-Field Slitless Spectroscopy data and returns a source catalog of all the low-mass stars in the field along with their calculated fundamental and secondary parameters.

### Requirements
- pip install SEDkit
- pip install bokeh

In [1]:
# Imports
from locals import source, catalog
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Making an SED for a single source

For each spectrum in our WFSS data products, a `Source` object is created. `locals` then searches Vizier to find ancillary photometry, parallaxes, and spectral types.

Let's pretend one of the brown dwarfs in our field is [LHS2924](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=LHS2924&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id).

In [2]:
# Coordinates of LHS 2924
ra = 217.180137*q.deg
dec = 33.177539*q.deg

# Make the source object
src = source.Source(ra=ra, dec=dec, name='LHS 2924')

# Find some photometry
src.find_photometry()

# Find a distance
src.find_parallax()

# Take a look
print('\n',src.photometry,'\n')

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Make this handle asymmetric uncertainties!

   band          eff         app_magnitude ...      abs_flux_unc      bandpass
                 um                       ... erg / (Angstrom cm2 s)         
-------- ------------------ ------------- ... ---------------------- --------
  Gaia.G 0.6597013110557894       16.6299 ...                    0.0   Gaia.G
 2MASS.J 1.2393093155660664 11.9899997711 ... 2.6969865361884258e-17  2MASS.J
 2MASS.H 1.6494947091246095 11.2250003815 ...   2.27922034211585e-17  2MASS.H
2MASS.Ks  2.163860596453316 10.7440004349 ... 1.3746130077877313e-17 2MASS.Ks
 WISE.W1 3.3897048577485647 10.4280004501 ...  3.622872277638672e-18  WISE.W1
 WISE.W2    4.6406356615375 10.1619997025 ... 1.3785181035558785e-18  WISE.W2
 WISE.W3 12.567593607521216 9.64599990845 ...   8.30331497963677e-20  WISE.W3
 WISE.W4 22.314225592308713 9.43700027466 ...  7.783593650024714e-20  WISE.W4 



Now we can calculate the fundamental parameters and plot the results.

In [3]:
src.results

Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


param,value,unc,units
object,object,object,object
name,LHS 2924,--,--
age,6.0,4.0,Gyr
distance,10.99,0.02,pc
parallax,90.9962,0.12710000574588776,mas
radius,0.8786209573091851,0.06782337214316517,solRad
spectral_type,--,--,--
membership,--,--,--
fbol,7e-11,7.88e-13,erg / (cm2 s)
mbol,13.905,0.012,--


In [4]:
fig = src.plot()

## Now make SEDs for all sources in the WFSS field
Now that we see the output of a single source, we can run this on the entire `SourceCatalog` generated from the WFSS output.

In [5]:
# Path to the JWST pipeline output directory
cat_path = resource_filename('locals', 'data/wfss/')

# Initialize empty source catalog
cat = catalog.SourceCatalog(cat_path)

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Make this handle asymmetric uncertainties!


All the sources are stored as `Source` instances within the `SourceCatalog.sources` attribute. 

For example, LHS 2924 is part of this catalog so let's make sure we get the same results as when it is run individually.

In [8]:
# Get the source instance
lhs2924 = cat.sources[1]

# Calculate the results
lhs2924.results

# Plot it
fig = lhs2924.plot()

Hooray!